# Chatbot with LangChain conversational chain and Llama2 🦙💬

이 노트북에서는 사업주의 정책과 같은 사용자 정의 데이터에 대한 질문에 응답할 수 있는 챗봇을 구축하겠습니다.

챗봇은 LangChain의 `ConversationalRetrievalChain`을 사용하며 다음과 같은 기능을 갖습니다.

- 자연어로 묻는 질문에 답변
- Elasticsearch에서 하이브리드 검색을 실행하여 질문에 답하는 문서를 찾으세요.
- Llama2 LLM을 활용하여 답변 추출 및 요약
- 후속 질문을 위한 대화 기억 유지

## Requirements 🧰

이 예에서는 다음이 필요합니다.

- Python 3.6 이상
- 로컬에 설치된 Elasticsearch
- OpenAI 계정

## Install packages 📦

먼저 이 예제에 필요한 패키지를 `pip install`합니다.


In [11]:
%pip install -U langchain openai elasticsearch tiktoken

## Initialize clients 🔌

다음으로 `getpass`를 사용하여 자격 증명을 입력합니다. `getpass`는 Python 표준 라이브러리의 일부이며 자격 증명을 안전하게 요청하는 데 사용됩니다.

In [12]:
from getpass import getpass

ES_URL = input('Elasticsearch URL(ex:https://127.0.0.1:9200): ')
ES_USER = "elastic" 
ES_USER_PASSWORD = getpass('elastic user PW: ')
CERT_PATH = input('Elasticsearch pem 파일 경로: ')
# pem 생성 방법: https://cdax.ch/2022/02/20/elasticsearch-python-workshop-1-the-basics/

# set OpenAI API key
# OPENAI_API_KEY = getpass("OpenAI API key")


## Load and process documents 📄

데이터를 로드할 시간입니다!   
우리는 직원 문서 및 정책 목록인 직장 검색 예제 데이터를 사용할 것입니다.


In [13]:
import json
from urllib.request import urlopen
import os

cwd = os.getcwd()
url = cwd + "/data/workplace-docs.json"
response = open(url)

workplace_docs = json.loads(response.read())

print(f"Successfully loaded {len(workplace_docs)} documents")

Successfully loaded 15 documents


## Chunk documents into passages 🪓

봇과 채팅하는 동안 봇은 관련 문서를 찾기 위해 인덱스에서 시멘틱 검색을 실행합니다.   
이것이 정확하려면 전체 문서를 작은 청크(chunk) -구절(passage)이라고도 함-로 분할해야 합니다.   
이런 방식으로 의미론적 검색은 문서 내에서 우리의 질문에 가장 답할 가능성이 높은 구절을 찾을 것입니다.

우리는 LangChain의 `CharacterTextSplitter`를 사용하고 문서의 텍스트를 청크 사이에 약간 겹치도록 800자로 분할할 것입니다.

In [14]:
from langchain.text_splitter import CharacterTextSplitter

metadata = []
content = []

for doc in workplace_docs:
    content.append(doc["content"])
    metadata.append({
        "name": doc["name"],
        "summary": doc["summary"]
    })

text_splitter = CharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=256
)
docs = text_splitter.create_documents(content, metadatas=metadata)

print(f"Split {len(workplace_docs)} documents into {len(docs)} passages")

Created a chunk of size 607, which is longer than the specified 512
Created a chunk of size 788, which is longer than the specified 512
Created a chunk of size 547, which is longer than the specified 512
Created a chunk of size 635, which is longer than the specified 512
Created a chunk of size 866, which is longer than the specified 512
Created a chunk of size 619, which is longer than the specified 512
Created a chunk of size 1120, which is longer than the specified 512
Created a chunk of size 567, which is longer than the specified 512


Split 15 documents into 89 passages


In [15]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_USER, ES_USER_PASSWORD),
    ca_certs=CERT_PATH
)

if client.indices.exists(index="workplace-docs"):
    client.indices.delete(index="workplace-docs")

임베딩을 생성하고 이를 사용하여 문서를 인덱싱해 보겠습니다.


In [16]:
from langchain.vectorstores import ElasticsearchStore
# from langchain.embeddings import OpenAIEmbeddings

# embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")

vector_store = ElasticsearchStore.from_documents(
    docs,
    es_connection = client,
    index_name="workplace-docs",
    embedding=embeddings
)

## Chat with the chatbot 💬

챗봇을 초기화해 보겠습니다.   
Elasticsearch를 문서 검색 및 채팅 세션 기록 저장을 위한 저장소로 정의하고,   
OpenAI를 질문을 해석하고 답변을 요약하는 LLM으로 정의한 다음, 이를 대화 체인에 전달합니다.

In [21]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

n_gpu_layers = None  # Metal set to 1 is enough.
n_batch = 1  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path = cwd + "/models/Llama-2-ko-7B-chat-gguf-q8_0.bin",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    n_ctx=2048,

    # https://www.reddit.com/r/LocalLLaMA/comments/1343bgz/what_model_parameters_is_everyone_using/
    temperature=0.7,
    top_k=2,
    top_p=0.1,

    max_tokens=4096,
    verbose=True,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [22]:
#from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from lib.elasticsearch_chat_message_history import ElasticsearchChatMessageHistory
from uuid import uuid4

retriever = vector_store.as_retriever()

# llm = OpenAI(openai_api_key=OPENAI_API_KEY)

chat = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

session_id = str(uuid4())
chat_history = ElasticsearchChatMessageHistory(
    client=vector_store.client,
    session_id=session_id,
    index="workplace-docs-chat-history"
)

이제 챗봇에 질문을 할 수 있습니다!

각 질문에 대한 컨텍스트로 채팅 기록이 어떻게 전달되는지 확인하세요.

In [23]:
# Define a convenience function for Q&A
def ask(question, chat_history):
    result = chat({"question": question, "chat_history": chat_history.messages})
    print(f"""[QUESTION] {question}
[ANSWER]  {result["answer"]}
          [SUPPORTING DOCUMENTS] {list(map(lambda d: d.metadata["name"], list(result["source_documents"])))}""")
    chat_history.add_user_message(result["question"])
    chat_history.add_ai_message(result["answer"])

# Chat away!
print(f"[CHAT SESSION ID] {session_id}")
ask("What does NASA stand for?", chat_history)
ask("Which countries are part of it?", chat_history)
ask("Who are the team's leads?", chat_history)
# ask("NASA는 무엇을 뜻하는가?", chat_history)
# ask("어떤 국가가 속해 있나요?", chat_history)
# ask("팀의 리더는 누구인가?", chat_history)

[CHAT SESSION ID] d91256bc-3ca6-45ad-840d-196f6ee73446
 National Aeronautics and Space Administration​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​[QUESTION] What does NASA stand for?
[ANSWER]   National Aeronautics and Space Administration​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​
          [SUPPORTING DOCUMENTS] ['Sales Organization Overview', 'Code Of Conduct', 'Code Of Conduct', 'New Employee Onboarding Guide']


Llama.generate: prefix-match hit


 What does NASA stand for?​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​
​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​​

Llama.generate: prefix-match hit


💡 _Try experimenting with other questions or after clearing the workplace data, and observe how the responses change._


# (Optional) Clean up 🧹

완료되면 이 세션의 채팅 기록을 정리할 수 있습니다

In [ ]:
chat_history.clear()

... or delete the indices.


In [ ]:
vector_store.client.indices.delete(index='workplace-docs')
vector_store.client.indices.delete(index='workplace-docs-chat-history')

ObjectApiResponse({'acknowledged': True})

: 